In [ ]:
%python3 -m venv .venv
%source .venv/bin/activate
%pip install -r requirements.txt

In [59]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = "chat_gpt_api_key"
MODEL = "gpt-3.5-turbo"
MODEL = "mixtral:8x7b"
MODEL = "llama2"

from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")


'Why was the math book sad? Because it had too many problems! 😂'

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

In [ ]:
from langchain_community.document_loaders import CSVLoader
import random

loader = CSVLoader("stock_data.csv")
pages = loader.load_and_split()
random_pages = random.sample(pages, 10)
random_pages

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

In [ ]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

In [44]:

from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(random_pages, embedding=embeddings)

In [ ]:

retriever = vectorstore.as_retriever()
retriever.invoke("MUNDRA")

In [48]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "What was the highest closing price in 2020?",
    "What is the average deliverable volume in 2011?",
    "What is the highest price in ADANIPORTS for the year 2030?",
    "what were the best 3 stocks in 2019 and why?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

In [ ]:
chain.invoke({'question': "but in the year 2025 no data is available for ADANIPORTS"})